# First Approach to Long-Short-Term Memory model #
 https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM


In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import permutations

from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf

import statsmodels.graphics.tsaplots as tsaplot
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense, Activation, Dropout, LSTM     # create layers


np.random.seed(42)
tf.random.set_seed(42)


We will start with the test train split 
in our case we can create several shorter sequences that we will use to train our model with 


In [36]:
df = pd.read_pickle("../data/final_dataframe.pkl")

In [37]:
def col_names(df):
    column_names = {'Photovoltaics [MWh] Original resolutions': 'Solar_generation_MWh',
                'Photovoltaics [MW] Calculated resolutions': 'Solar_installed_MW',
                'Total (grid load) [MWh] Original resolutions': 'Total_consumption_MWh',
                'Germany/Luxembourg [€/MWh] Calculated resolutions': 'DE_LU_price_per_MWh',}
    df.rename(columns=column_names, inplace=True)


In [38]:
col_names(df)

In [39]:
df.set_index('Date')

,Solar_generation_MWh,Solar_installed_MW,Total_consumption_MWh,DE_LU_price_per_MWh,normalisation_factor,Solar_generation_MWh_normalized
Date,,,,,,
2018-10-01 00:00:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 00:15:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 00:30:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 00:45:00,0.0,42805.0,10589.75,59.53,0.684015,0.0
2018-10-01 01:00:00,0.0,42805.0,10589.75,56.10,0.684015,0.0
...,...,...,...,...,...,...
2023-06-01 22:45:00,0.0,62579.0,12945.50,95.41,1.000000,0.0
2023-06-01 23:00:00,0.0,62579.0,12817.75,86.53,1.000000,0.0
2023-06-01 23:15:00,0.0,62579.0,12539.00,86.53,1.000000,0.0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163680 entries, 0 to 163679
Data columns (total 7 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Date                             163680 non-null  datetime64[ns]
 1   Solar_generation_MWh             163680 non-null  float64       
 2   Solar_installed_MW               163680 non-null  float64       
 3   Total_consumption_MWh            163680 non-null  float64       
 4   DE_LU_price_per_MWh              163680 non-null  float64       
 5   normalisation_factor             163680 non-null  float64       
 6   Solar_generation_MWh_normalized  163680 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 8.7 MB


In [6]:
# We have a lot of samples therefore I will limit the sample size for training a bit mor
#df = df.iloc[60000: , :]
#len(df)

In [44]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df[['Solar_generation_MWh_normalized']], test_size=.25, shuffle=False)

In [45]:
#Let's scale the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [46]:
#! Work in progress ...

scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [47]:
# split a univariate sequence into samples
def split_sequence(input, n_steps, pred_size):
    x, y = list(), list()
    for i in range(len(input)):
        end_ix = i + n_steps # find the end of this pattern
        if end_ix+pred_size > len(input)-1: # check if we are beyond the sequence
            break
        seq_x, seq_y = input[i:end_ix], input[end_ix: end_ix+pred_size]# gather input and output parts of the pattern
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [48]:
# define input sequence
input = train
# choose a number of time steps
n_steps = 672

# prediction size 
pred_size= 96
# split into samples
X, y = split_sequence(input, n_steps, pred_size)
# summarize the data
print(len(X), len(y))


121992 121992


In [49]:
print(y.shape, X.shape)

(121992, 96, 1) (121992, 672, 1)


In [51]:
#Now we have to define the validation set for our model #! I see this approach is not so useful, therfore I will use the train test split with shuffling to obtain the validation data. Here i am not loosing the lateest data for training my model 
def val_set(X,y):
    X, X_val, y, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)
    return X, X_val, y, y_val
    #! old approach
    #train_size = round(len(X) * 0.8)
    #X = X[:train_size, :]
    #X_val = X[train_size:, :]
    #y = y[:train_size, :]
    #y_val = y[train_size:, :]
    
X, X_val, y, y_val = val_set(X, y)

In [52]:
X.shape

(97593, 672, 1)

In [53]:
# reshape from [samples, timesteps] into [samples, timesteps, features]

def reshape_for_LSTM(X, y, features):
    features
    X = X.reshape((X.shape[0], X.shape[1], features))
    y = y.reshape((y.shape[0], y.shape[1]))
    return X, y

In [54]:
X, y = reshape_for_LSTM(X, y, 1)

In [55]:
X_val, y_val = reshape_for_LSTM(X_val, y_val, 1)

In [56]:
X_val.shape

(24399, 672, 1)

In [57]:
X.shape

(97593, 672, 1)

## Lets start the modeling approach using the Long short term memory model ##



In [58]:
# Define dictionary to store results
history = {}

# Define number of epochs and learning rate decay
N_TRAIN = len(X)
EPOCHS = 50
BATCH_SIZE = 2371 # total sample size = 113808 each batch 2371 samples (48 batches ) #! has to be adjusted further to improve
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(  #! check the source 
    0.01,  #! please adjust and finetune
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)


# Define optimizer used for modelling
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule, name='Adam')  # due to a warning message I used the legacy.Adam 

In [61]:
# Define path where checkpoints should be stored
checkpoint_path = "modeling/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0) # Set verbose != 0 if you want output during training 

#create a callback to stop early once there is no improvement in the loss
cp_early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True,
                                verbose = True)

Note how many output layer are needed for predicting several timestamps? Please check one output layer is enough but some of the parameters have to be adjusted,

n_steps, n_features
X.shape[1], X.shape[2]

reason for not having activation functions https://datascience.stackexchange.com/questions/66594/activation-function-between-lstm-layers
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTMCell

output layer structure : https://stackoverflow.com/questions/46797891/output-shape-of-lstm-model#46799544

https://shiva-verma.medium.com/understanding-input-and-output-shape-in-lstm-keras-c501ee95c65e

In [62]:
def get_simple_LSTM_model():
    simple_LSTM = tf.keras.Sequential([
      tf.keras.layers.LSTM(45, kernel_initializer = 'uniform', input_shape = (X.shape[1], X.shape[2]), return_sequences=True), # ! units are not set in stone yet 
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.LSTM(32, return_sequences=False),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(y.shape[1] ,kernel_initializer = 'uniform', activation='linear') #96 to predict a day 
    ])

    simple_LSTM.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanAbsolutePercentageError(), 
                  metrics=[tf.keras.losses.MeanAbsolutePercentageError()])
    return simple_LSTM

In [77]:
with tf.device('/cpu:0'):
    simple_LSTM = get_simple_LSTM_model()
    print(simple_LSTM.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 672, 45)           8460      
                                                                 
 dropout_4 (Dropout)         (None, 672, 45)           0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                9984      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 96)                3168      
                                                                 
Total params: 21612 (84.42 KB)
Trainable params: 21612 (84.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [64]:
with tf.device('/cpu:0'):
    history = simple_LSTM.fit(X,
                        y,
                        batch_size= BATCH_SIZE,
                        validation_data= (X_val, y_val),   ##### probably best to make validation data D #! TO DO 
                        verbose=100,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS,
                        callbacks=[cp_callback, cp_early_stop])

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Restoring model weights from the end of the best epoch: 12.
Epoch 13: early stopping


In [65]:
# first we split the test set too 
X_test, y_test = split_sequence(test, n_steps, pred_size)


In [66]:
history.history

{'loss': [100.73613739013672,
  98.66854095458984,
  96.99805450439453,
  95.61962890625,
  91.35282897949219,
  85.80809783935547,
  81.61038970947266,
  77.80121612548828,
  74.92119598388672,
  72.56488800048828,
  72.50821685791016,
  69.57533264160156,
  68.83611297607422],
 'mean_absolute_percentage_error': [100.73612213134766,
  98.64579010009766,
  96.99183654785156,
  95.58751678466797,
  91.35536193847656,
  85.86908721923828,
  81.67713165283203,
  77.75575256347656,
  74.8548812866211,
  72.54803466796875,
  72.52290344238281,
  69.5459213256836,
  68.76153564453125],
 'val_loss': [99.49559783935547,
  96.99715423583984,
  96.01146697998047,
  91.47895050048828,
  86.82050323486328,
  78.06043243408203,
  75.76201629638672,
  72.53741455078125,
  68.71493530273438,
  66.8681411743164,
  65.646484375,
  63.96110534667969,
  64.04865264892578],
 'val_mean_absolute_percentage_error': [99.62149047851562,
  96.88880157470703,
  95.9996337890625,
  91.46997833251953,
  86.6918563

In [78]:
X_test[:]

array([[[-0.32100538],
        [-0.3898191 ],
        [-0.45411551],
        ...,
        [ 0.17424262],
        [ 0.07983079],
        [-0.0069016 ]],

       [[-0.3898191 ],
        [-0.45411551],
        [-0.51163595],
        ...,
        [ 0.07983079],
        [-0.0069016 ],
        [-0.09513977]],

       [[-0.45411551],
        [-0.51163595],
        [-0.55515273],
        ...,
        [-0.0069016 ],
        [-0.09513977],
        [-0.18066754]],

       ...,

       [[-0.62095491],
        [-0.62095491],
        [-0.62095491],
        ...,
        [-0.62095491],
        [-0.62095491],
        [-0.62095491]],

       [[-0.62095491],
        [-0.62095491],
        [-0.62095491],
        ...,
        [-0.62095491],
        [-0.62095491],
        [-0.62095491]],

       [[-0.62095491],
        [-0.62095491],
        [-0.62095491],
        ...,
        [-0.62095491],
        [-0.62095491],
        [-0.62095491]]])

than we take only the first element of the splited test set and let the model predict 

In [79]:
y_test = y_test[:2, :]

In [80]:
y_test

array([[[-0.09513977],
        [-0.18066754],
        [-0.26423782],
        [-0.3484104 ],
        [-0.43604626],
        [-0.50711864],
        [-0.5697587 ],
        [-0.60830643],
        [-0.61899741],
        [-0.62080433],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0.62095491],
        [-0

In [81]:
x_input = X_test[:2, :]
x_input = x_input.reshape((x_input.shape[0], x_input.shape[1], 1))
y_pred = simple_LSTM.predict(x_input, verbose=0)
print(y_pred)

[[-4.7208104e-04  9.7919255e-04  9.4636454e-04 -1.9382272e-03
  -1.0061392e-03  4.2466763e-03 -2.4395650e-03 -4.3593952e-03
   1.5452302e-03  4.7673211e-03  1.6838755e-04 -2.0142121e-03
  -1.0609244e-03 -2.5246793e-03  3.5162102e-03 -2.4984281e-03
  -1.8573541e-04 -2.6503194e-03  1.2697493e-04 -2.2307732e-04
  -1.2507252e-03 -1.0704120e-03  9.8839548e-05  2.0514510e-03
   9.7309443e-04 -3.1822280e-03 -1.1050068e-03 -1.8370979e-03
   5.2274838e-03  5.5532041e-04  2.4710987e-03 -2.6858896e-03
   6.5505359e-04 -7.4892188e-04 -7.0136535e-04  1.7484848e-03
  -9.4406918e-04 -2.2345120e-03  8.0291618e-04  1.4746258e-03
  -1.9499791e-03  3.8378488e-04  3.4212316e-03 -3.3996433e-03
  -2.2145938e-03  4.7899014e-03 -4.9913314e-04 -1.8754652e-03
  -2.8572392e-03  6.1539892e-04  1.6520181e-03 -1.2593172e-04
   1.7499775e-03 -1.1104214e-03  2.4176082e-03 -1.7283756e-03
  -8.2828839e-05 -3.0896852e-03  2.1329785e-03 -2.4194177e-03
   5.0876458e-04  6.4426399e-04 -2.1464569e-03 -3.1307290e-04
  -1.101

In [73]:
def last_mse(y_test, y_pred):
    return keras.metrics.mean_absolute_percentage_error(y_test.reshape(y_test.shape[0], y_test.shape[1]), y_pred)

rmse = mean_squared_error(y_test, y_pred)

In [74]:
last_mse(y_test, y_pred)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([54.057102, 55.348057], dtype=float32)>

In [76]:
# Save the entire small model as a SavedModel.
!mkdir -p ../models/saved_model
simple_LSTM.save('../models/saved_model/simple_LSTM')

INFO:tensorflow:Assets written to: ../models/saved_model/simple_LSTM/assets


INFO:tensorflow:Assets written to: ../models/saved_model/simple_LSTM/assets


### Short summary ###
model is performing shitty it looks like it cannot handle the seasonality at all 

### Alternative approach get rif of the seasonality beforehand 

In [ ]:
df['Solar_detrend'] = (df['Solar_generation_MWh_normalized'] - df['Solar_generation_MWh_normalized'].shift(96))


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163680 entries, 0 to 163679
Data columns (total 8 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Date                             163680 non-null  datetime64[ns]
 1   Solar_generation_MWh             163680 non-null  float64       
 2   Solar_installed_MW               163680 non-null  float64       
 3   Total_consumption_MWh            163680 non-null  float64       
 4   DE_LU_price_per_MWh              163680 non-null  float64       
 5   normalisation_factor             163680 non-null  float64       
 6   Solar_generation_MWh_normalized  163680 non-null  float64       
 7   Solar_detrend                    163584 non-null  float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 10.0 MB


In [ ]:
#Split into test and train set
train, test = train_test_split(df['Solar_detrend'], test_size=.3, shuffle=False)

# define input sequence
input = train
# choose a number of time steps (a week)
n_steps = 672

# prediction size (we want to predict a day)
pred_size= 96

# split into samples
X, y = split_sequence(input, n_steps, pred_size)
# summarize the data
print(len(X), len(y))

113808 113808


In [ ]:
# split the train set into train and validation set
X, X_val, y, y_val = val_set(X, y)

In [ ]:
X, y = reshape_for_LSTM(X, y, 1)

In [ ]:
def get_season_LSTM_model():
    wo_season_LSTM = tf.keras.Sequential([
      tf.keras.layers.LSTM(45 ,kernel_initializer = 'uniform', input_shape = (X.shape[1], X.shape[2]), return_sequences=True), # ! units are not set in stone yet 
      tf.keras.layers.LSTM(32, return_sequences=False),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(y.shape[1] ,kernel_initializer = 'uniform', activation='relu') #96 to predict a day 
    ])

    wo_season_LSTM.compile(optimizer=optimizer,
                  loss='mean_squared_error')
    return wo_season_LSTM

In [ ]:
with tf.device('/cpu:0'):
    wo_season_LSTM = get_season_LSTM_model()
    print(wo_season_LSTM.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 672, 45)           8460      
                                                                 
 lstm_13 (LSTM)              (None, 32)                9984      
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 96)                3168      
                                                                 
Total params: 21612 (84.42 KB)
Trainable params: 21612 (84.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
with tf.device('/cpu:0'):
    training_history['season_free'] = wo_season_LSTM.fit(X,
                        y,
                        batch_size= BATCH_SIZE,
                        validation_data= (X_val, y_val),   ##### probably best to make validation data D #! TO DO 
                        verbose=100,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=EPOCHS,
                        callbacks=[cp_callback, cp_early_stop])

Epoch 1/200
Epoch 2/200


In [ ]:
training_history['season_free'].history

{'loss': [nan, nan], 'mse': [nan, nan]}

# short Summary 
sadly I still only get nan for loss and mse. check with this in detail 
https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network

In [ ]:
print("Evaluate on test data")
results = wo_season_LSTM.evaluate(X_test, y_test, batch_size=2371)
print("test loss, test acc:", results)

Evaluate on test data


ValueError: Data cardinality is ambiguous:
  x sizes: 48336
  y sizes: 2
Make sure all arrays contain the same number of samples.